In [ ]:
from data_loader.MTL_dataloader import get_train_validation_generator
from utils.ModelBlock import ModelBlock
from utils.Encoder import Encoder
from utils.Detector import Detector
from utils.Segmenter import Segmenter
from utils.Classifier import Classifier
from utils.loss.MTL_loss import MTL_loss
from tensorflow.keras.optimizers import Adam

In [ ]:
det_csv_path = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
seg_csv_path = "/kaggle/input/siim-acr-pneumothorax-segmentation-data/train-rle.csv"
det_images_path = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/"
seg_images_path = "/kaggle/input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/"

In [ ]:
train_gen,val_gen = get_train_validation_generator(det_csv_path,seg_csv_path , det_images_path, seg_images_path)

In [ ]:
encoder = Encoder( )

In [ ]:
classifier = Classifier(encoder)

In [ ]:
img_size = 256
n_classes=1
detector=Detector(encoder,img_size, n_classes)

In [ ]:
segmenter = Segmenter(encoder)

In [ ]:
MTL_model = ModelBlock.add_heads(encoder,[classifier,detector,segmenter ])

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(MTL_model)

In [ ]:
#MTL_model.summary()

In [ ]:
classification_loss= "categorical_crossentropy"
detection_loss= detector.loss
segmentation_loss= segmenter.loss
mtl_loss= MTL_loss(classification_loss , detection_loss ,segmentation_loss)
lossWeights = [1.0,1.0,1.0]

In [ ]:
INIT_LR = 1e-4
EPOCHS =20

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

In [ ]:
MTL_model.compile(optimizer=opt, loss=mtl_loss, metrics=[])

In [ ]:
MTL_model.fit_generator(train_gen,validation_data=val_gen)